#### COINTEGRATION FILTER TO FIND STOCKS   
- Goal: Find only the most stable, highly significant pairs
- Risk: Fewer opportunities, more selective

Parameters:
- Correlation threshold: 0.98
- P-value threshold: 0.01
- ADF Statistic threshold:  loweest 10 pairs. 
- Time period: 15 years
- Min correlation with individual correlations > 0.98

In [32]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy import stats
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import adfuller
import warnings
from GetFreshMarketData import *
from tqdm import tqdm
import polars as pl
tqdm.pandas()
warnings.filterwarnings('ignore')

In [ ]:
INDEX_NAME = "NIFTY_500"
index_symbols  = pd.read_csv(fr"C:\Users\ksdee\Documents\Trading\Data\index\constituents\{INDEX_NAME}.csv")
index_symbols = index_symbols.loc[((index_symbols.symbol!='DUMMYHDLVR') ),:] 
index_symbols = index_symbols.loc[index_symbols.series == 'EQ','symbol'].to_list()

end_date = datetime.today()
start_date = end_date - timedelta(days = 365 * 1)
corr_threshold = 0.7

stock_dict = {}
for sym in tqdm(index_symbols):
    file = STOCK_DIR/f'{sym}.csv'
    if file.exists():
        df = pd.read_csv(file, low_memory=False)
        df.date = df.date.apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
        df = df.loc[df.date >= start_date,:]
        df = df.sort_values(by='date')
        stock_dict[sym] = df

stock_pairs = list(combinations(index_symbols, 2))
combinations_df = pd.DataFrame(stock_pairs, columns=['Stock_1', 'Stock_2'])


100%|██████████| 499/499 [00:22<00:00, 22.58it/s]


In [25]:
def generate_corr(row):
    stock1 = row['Stock_1']
    stock2 = row['Stock_2']
    df1 = stock_dict[stock1]
    df2 = stock_dict[stock2]
    if df1.date.min() >= df2.date.min():
        df1 = df1.loc[df1.date.isin(df2.date),:]
    else:
        df2 = df2.loc[df2.date.isin(df1.date),:]

    is_equal = np.array_equal(df1.date.values, df2.date.to_numpy())
    if is_equal:
        df1 = df1.set_index('date')
        df1 = df1.sort_index()
        df2 = df2.set_index('date')
        df2 = df2.sort_index()

        df1['returns'] = np.log(df1.close/df1.close.shift(1))
        df2['returns'] = np.log(df2.close/df2.close.shift(1))
        correlation = df1.returns.dropna().corr(df2.returns.dropna())


    else:
        correlation = np.nan
    return correlation
combinations_df['Correlation']=combinations_df.progress_apply(lambda x: generate_corr(x),axis=1)


100%|██████████| 124251/124251 [06:03<00:00, 341.68it/s]


In [36]:
combinations_df = combinations_df.dropna(subset='Correlation')
combinations_df = combinations_df.loc[combinations_df.Correlation.apply(lambda x : abs(x))>corr_threshold,:]

In [41]:
def find_coint(row):
    stock1 = row['Stock_1']
    stock2 = row['Stock_2']
    df1 = np.log(stock_dict[stock1].close)
    df2 = np.log(stock_dict[stock2].close)
    score, p_value, crit_values = coint(df1, df2)
    return score,p_value,crit_values

combinations_df[['score','p_value','crit_values']]=combinations_df.progress_apply(lambda x: pd.Series(find_coint(x)),axis=1)
combinations_df = combinations_df.loc[combinations_df.p_value<0.05,:]


100%|██████████| 76/76 [00:00<00:00, 83.49it/s]


In [46]:
def build_reg(row):
    stock1 = row['Stock_1']
    stock2 = row['Stock_2']
    y = np.log(stock_dict[stock1].close)
    x = np.log(stock_dict[stock2].close)
    x_with_constant = add_constant(x)
    model = OLS(y.values, x_with_constant.values).fit()

    alpha = model.params[0]  # Intercept
    beta = model.params[1]   # Hedge Ratio (Slope)

    return alpha, beta

combinations_df[['alpha','beta']]=combinations_df.progress_apply(lambda x: pd.Series(build_reg(x)),axis=1)

100%|██████████| 6/6 [00:00<00:00, 163.42it/s]


In [58]:
def find_z_score(row):
    stock1 = row['Stock_1']
    stock2 = row['Stock_2']
    alpha = row['alpha']
    beta = row['beta']

    y = np.log(stock_dict[stock1].close)
    x = np.log(stock_dict[stock2].close)
    spread = y - (beta * x + alpha)

    window = 10
    rolling_mean = spread.rolling(window=window).mean()
    rolling_std = spread.rolling(window=window).std()
    z_score = (spread - rolling_mean) / rolling_std
    return z_score.values[-1]

combinations_df['z_score'] = combinations_df.progress_apply(lambda x: find_z_score(x),axis=1)


100%|██████████| 6/6 [00:00<00:00, 292.49it/s]


In [59]:
combinations_df

,Stock_1,Stock_2,Correlation,score,p_value,crit_values,alpha,beta,z_score
68534,FACT,RCF,0.743421,-3.900117,0.009916,"[-3.941513937471082, -3.361080551920153, -3.06...",-0.945985,1.548330,0.210856
83237,HINDPETRO,IOC,0.772281,-3.574337,0.026350,"[-3.941513937471082, -3.361080551920153, -3.06...",0.290513,1.156078,-0.871141
91756,IRFC,RVNL,0.851902,-3.492601,0.033057,"[-3.941513937471082, -3.361080551920153, -3.06...",1.001529,0.654578,NaN
98092,JINDALSTEL,TATASTEEL,0.705208,-4.605631,0.000817,"[-3.941513937471082, -3.361080551920153, -3.06...",3.302657,0.704997,-0.796396
111671,NATIONALUM,VEDL,0.731778,-3.794313,0.013794,"[-3.941513937471082, -3.361080551920153, -3.06...",-4.006561,1.525571,-2.101095
121497,SAIL,TATASTEEL,0.764272,-3.509256,0.031584,"[-3.941513937471082, -3.361080551920153, -3.06...",0.519652,0.852266,-1.229113
